In [1]:
import sys
sys.path.insert(1, "./rnn")
import train
import model


In [2]:
from FileReader import FileReader
classInstance = FileReader
df = classInstance.get_dataframe() #IMPORTANT:  saves a pickle to output/simple or output/full. 
import sys
sys.path.insert(1, "./feature-extraction/embed-extractor")
#from EmbedExtractor import EmbedExtractor
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json
#ee = EmbedExtractor()





 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 544294893146091520

 768859780240773121

 764927075522260992

 544274544174071809

 498486826269548545

 544315472075042818

 524923293711998976

 769988636754505729

 524936793633083394

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 54429489314

In [3]:
# need stuff from main.py to test transformation into Tensors

def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    #word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    #df['wordEmbedding'] = word_embeddings


    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 0
    df.loc[df.classification == 'unverified', 'classification'] = 2
    # getting the labels

    #removed containsURL
    attributes = ['isVulgar', 'containsAdjective', 'containsEmoji', 'containsAbbreviation', 'wordCount']
    for tag in TwitterParser.tagset:
        attributes.append('num_' + tag)

    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    #word_embedding_values = df['wordEmbedding'].values.tolist()


    #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
    ###for i,index in enumerate(df.index):
    ###    dev_values[i].append(int(index))


# UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    #for i,d in enumerate(word_embedding_values):
     #   values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values

    return labels, values


In [4]:
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/dev_data_simple.pickle', index=1)

In [24]:
## convert to tensor Variable here
import tensorflow as tf
import torch
import torch.nn as nn
import helpers
t = helpers.convert_to_tensor(tr_values)

In [26]:
n_epochs = 2000
print_every = 100
hidden_size = 50
learning_rate = 0.01
chunk_len = 200


decoder = model.RNN(input_size = 30, hidden_size=hidden_size, output_size = 30)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), learning_rate)
# Loss function can be changed as an argument?

criterion = nn.CrossEntropyLoss()


In [28]:
#from the pytorch tutroial
try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in range(1, n_epochs + 1):
        loss = train(train.random_training_set(chunk_len))
        loss_avg += loss

        if epoch % args.print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, 'Wh', 100), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()
   

for v in dev_values():
    predictions = model.predict(dev_values)
    
print(predictions)






Training for 2000 epochs...


NameError: name 'file_len' is not defined